In [12]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.vgg19 import preprocess_input
from keras.applications.vgg19 import VGG19
import tensorflow as tf
from PIL import Image, ImageFile
import PIL.Image
import numpy as np

ImageFile.LOAD_TRUNCATED_IMAGES = True

# Data

Our data will is in the DATASET folder split into our TRAIN and TEST datasets. Each folder contains one folder for each of our labels. Our next step will be to preprocess our data.

- We use ImageDataGenerator to do some transformations of the images
- Then we can apply the transformations to the directories for our train and test folder


In [6]:
train_dir = "DATASET/TRAIN"
test_dir = "DATASET/TEST"


In [7]:
train_datagen = ImageDataGenerator(
    width_shift_range=0.1,
    horizontal_flip=True,
    rescale=1.0 / 255,
    validation_split=0.2,
    preprocessing_function=preprocess_input,
)
test_datagen = ImageDataGenerator(
    rescale=1.0 / 255, preprocessing_function=preprocess_input
)

In [8]:
train_generator = train_datagen.flow_from_directory(
    directory=train_dir,
    target_size=(48, 48),
    color_mode="rgb",
    class_mode="categorical",
    batch_size=16,
    subset="training",
)
validation_generator = train_datagen.flow_from_directory(
    directory=train_dir,
    target_size=(48, 48),
    color_mode="rgb",
    class_mode="categorical",
    batch_size=16,
    subset="validation",
)

Found 22968 images belonging to 7 classes.
Found 5741 images belonging to 7 classes.


In [9]:
print(validation_generator.class_indices)
print(train_generator.class_indices)

{'angry': 0, 'disgust': 1, 'fear': 2, 'happy': 3, 'neutral': 4, 'sad': 5, 'surprise': 6}
{'angry': 0, 'disgust': 1, 'fear': 2, 'happy': 3, 'neutral': 4, 'sad': 5, 'surprise': 6}


In [13]:
train_generator.classes.count(6)


AttributeError: 'numpy.ndarray' object has no attribute 'count'

In [16]:
unique, counts = np.unique(train_generator.classes, return_counts=True)
classes_count = dict(zip(unique, counts))
print(classes_count)

{0: 3196, 1: 349, 2: 3278, 3: 5772, 4: 3972, 5: 3864, 6: 2537}


In [23]:
# Determine the number of samples in the smallest class
num_samples_per_class = min(classes_count.values())

# Use class weights to balance the dataset during training
class_weights = {
    c: num_samples_per_class / classes_count[c]
    for c in range(train_generator.num_classes)
}

print(class_weights)

{0: 0.10919899874843554, 1: 1.0, 2: 0.1064673581452105, 3: 0.060464310464310465, 4: 0.087865055387714, 5: 0.09032091097308488, 6: 0.13756405202995664}


We will use the VGG19 model you can read more about the requirements and considerations for this model in the documentation (https://keras.io/api/applications/vgg/).


In [24]:
model = VGG19(include_top=False, weights="imagenet", input_shape=(48, 48, 3))

# Freeze the imported layers so they cannot be retrained.
for layer in model.layers:
    layer.trainable = False


model.summary()


Model: "vgg19"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 48, 48, 3)]       0         
                                                                 
 block1_conv1 (Conv2D)       (None, 48, 48, 64)        1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 48, 48, 64)        36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 24, 24, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 24, 24, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 24, 24, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 12, 12, 128)       0     

### Adding flattening and dense layers

Right now, our model is missing a top to actually classify our features. Let's add them:


In [25]:
from keras import Sequential
from keras.layers import Dense
from keras.layers import Flatten

new_model = Sequential()
new_model.add(model)
new_model.add(Flatten())
new_model.add(Dense(7, activation="softmax"))

# Summarize.
new_model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg19 (Functional)          (None, 1, 1, 512)         20024384  
                                                                 
 flatten (Flatten)           (None, 512)               0         
                                                                 
 dense (Dense)               (None, 7)                 3591      
                                                                 
Total params: 20,027,975
Trainable params: 3,591
Non-trainable params: 20,024,384
_________________________________________________________________


In [29]:
from tensorflow.keras.optimizers import Adam

# Compile and fit the model. Use the Adam optimizer and crossentropical loss.
# Use the validation data argument during fitting to include your validation data.
optimizer = Adam(learning_rate=0.0001)
new_model.compile(
    optimizer=optimizer, loss="categorical_crossentropy", metrics=["accuracy"]
)
history = new_model.fit(
    train_generator,
    epochs=5,
    batch_size=16,
    validation_data=validation_generator,
    class_weight=class_weights,
)

Epoch 1/5
1436/1436 [==============================] - 215s 149ms/step - loss: 0.2074 - accuracy: 0.1820 - val_loss: 1.8937 - val_accuracy: 0.2177
Epoch 2/5
1436/1436 [==============================] - 116s 81ms/step - loss: 0.1961 - accuracy: 0.2465 - val_loss: 1.8314 - val_accuracy: 0.2693
Epoch 3/5
1436/1436 [==============================] - 119s 83ms/step - loss: 0.1917 - accuracy: 0.2734 - val_loss: 1.8224 - val_accuracy: 0.2716
Epoch 4/5
1436/1436 [==============================] - 123s 85ms/step - loss: 0.1893 - accuracy: 0.2865 - val_loss: 1.7994 - val_accuracy: 0.2883
Epoch 5/5
1436/1436 [==============================] - 122s 85ms/step - loss: 0.1879 - accuracy: 0.2926 - val_loss: 1.8165 - val_accuracy: 0.2669


# Predicting the class of your image

Let's take this bad boy for a spin! Can your image get properly identified?


In [30]:
import numpy as np
from tensorflow.keras.applications.vgg19 import preprocess_input

# Predict the class of your picture.

img = tf.keras.preprocessing.image.load_img(
    "./test_folder/maité.png", target_size=(48, 48)
)


img_nparray = tf.keras.preprocessing.image.img_to_array(img)

print(img_nparray.shape)
# convert image to array

x = preprocess_input(img_nparray).reshape((1, 48, 48, 3))

print(x.shape)

prediction = new_model.predict(x)

print(prediction.shape)

# create a list containing the class labels
# class_labels = ["downdog", "goddess", "plank", "tree", "warrior2"]
class_labels = list(validation_generator.class_indices.keys())
print(class_labels)

# find the index of the class with maximum score
pred = np.argmax(prediction, axis=-1)
class_labels[pred[0]]

(48, 48, 3)
(1, 48, 48, 3)
1/1 [==============================] - 0s 252ms/step
(1, 7)
['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']


'neutral'